In [1]:
import geopandas as gpd
import folium
from folium import FeatureGroup, LayerControl
import glob
import os
import pandas as pd

In [2]:
# Define the directory containing shapefiles
shapefiles_dir = 'data'  # Change this to your folder path if different

# Use glob to find all .shp files in the directory
shp_files = glob.glob(os.path.join(shapefiles_dir, "*.shp"))

print(f"Found {len(shp_files)} shapefiles.")

Found 2 shapefiles.


In [3]:
# Initialize an empty list to store GeoDataFrames
gdf_list = []

for shp in shp_files:
    try:
        gdf = gpd.read_file(shp)
        gdf_list.append(gdf)
        print(f"Loaded: {shp}")
    except Exception as e:
        print(f"Error loading {shp}: {e}")

target_crs = "EPSG:4326"  # WGS84

gdf_list = [gdf.to_crs(target_crs) for gdf in gdf_list]

# Concatenate all GeoDataFrames into one
if gdf_list:
    consolidated_gdf = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True))
    print("Successfully consolidated shapefiles.")
else:
    raise ValueError("No shapefiles were loaded. Please check the directory path and file formats.")

Loaded: data\SedimentaryBasins_US_May2011_v2.shp
Loaded: data\ShalePlays_US_EIA_Dec2021.shp
Successfully consolidated shapefiles.


In [4]:
# Create an interactive map
centroid = consolidated_gdf.unary_union.centroid
m = folium.Map(location=[centroid.y, centroid.x], zoom_start=10)

for gdf, shp in zip(gdf_list, shp_files):
    geojson = gdf.to_json()
    feature_group = FeatureGroup(name=os.path.basename(shp))
    
    # Define the fields to include in the tooltip, excluding 'geometry'
    tooltip_fields = [col for col in gdf.columns if col != 'geometry']
    tooltip_aliases = [col.replace('_', ' ').title() for col in tooltip_fields]
    
    # Create and add the GeoJson layer with the customized tooltip
    folium.GeoJson(
        geojson,
        name=os.path.basename(shp),
        tooltip=folium.GeoJsonTooltip(
            fields=tooltip_fields,
            aliases=tooltip_aliases,
            localize=True
        )
    ).add_to(feature_group)
    
    # Add the FeatureGroup to the map
    feature_group.add_to(m)

LayerControl().add_to(m)

# Display the map in the notebook
m

In [5]:
# Save the map as HTML
output_map = 'consolidated_map.html'
m.save(output_map)
print(f"Map has been saved to {output_map}")

# Export the consolidated shapefile
output_shp = 'consolidated_shapefile.shp'

if consolidated_gdf.crs is None:
    consolidated_gdf.set_crs(epsg=4326, inplace=True)

consolidated_gdf.to_file(output_shp)
print(f"Consolidated shapefile has been saved to {output_shp}")

Map has been saved to consolidated_map.html
Consolidated shapefile has been saved to consolidated_shapefile.shp
